In [ ]:
# Databricks notebook source
# Balanceamento de Carga , quando uma ou mais partições recebem mais carga/quantidade de dados, processo conhecido como Skewness/assimetria de dados
from pyspark.sql.functions import col
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Criando os Dataframes de exemplo
data1 = [("A",1),("B",2),("C",3),("D",4)]
data2 = [("A",100),("A",101),("A",102),("B",200),("C",300)]

In [ ]:
df1 = spark.createDataFrame(data1, ["Key","Value"])
df2 = spark.createDataFrame(data2, ["Key","Value"])

In [ ]:
# Join que causa skewness
joined_df = df1.join(df2,"Key")

Exibindo os resultados

In [ ]:
joined_df.show()

COMMAND ----------

Contando as ocorrências de cada chave

In [ ]:
df2_count = df2.groupBy("Key").count()

Convertendo para pandas DataFrame para visualização em gráficos via matplotlib

In [ ]:
pandas_df2_count = df2_count.toPandas()

In [ ]:
# Criando um gráfico de barras, para visualizar a contagem do dataframe pandas_df2_count
plt.figure(figsize= (8,6))
plt.bar(pandas_df2_count["Key"],pandas_df2_count["count"] ) # eixo x e eixo y
plt.xlabel ("Key") # Lable do eixo x
plt.ylabel("Count") # Lable do eixo y
plt.title("Distribuição dos Dados Antes de realizar o Join")
plt.show()

COMMAND ----------

MAGIC %md
MAGIC Utilizando a TÉCNICA REPARTITION
MAGIC

COMMAND ----------

tamanho dos dados / ambiente da aplicação = número de partições
Reparticionamento utilzando função repartition, 
informa o número de partições
verificar a distribuição dos dados. verificar a contagem da memória, o tamanho do arquivo , a quantidade de partições deve ser múltiplo em relação ao tamanho dos cores.
esse comando faz o shuffle, reposiciona os dados entre os nós dos clusters
operação + cara

In [ ]:
df1_repartitioned = df1.repartition(2,'Key') # número de partições / 'Campo utilizado para a chave'
df2_repartitioned = df2.repartition(2,'Key') # número de partições / 'Campo utilizado para a chave'

In [ ]:
joined_df = df1_repartitioned.join(df2_repartitioned,'Key')

In [ ]:
joined_df.show()

COMMAND ----------

MAGIC %md
MAGIC Usando a técnica SALTING
MAGIC

COMMAND ----------

o salting, são valores aleatórios, que podem ser usados como chave de particionamento, facilita a identificação única dos registros, quando o join é realizado
de maneira aleatória ( randomica função rand(), ele acrescenta valores numéricos a nova coluna criada)
Risco: quando o dataframe menor é transformado em um dataframe maior , deve verificar se a performance melhora, caso contrário, usar outra técnica

In [ ]:
from pyspark.sql.functions import col, rand
import pyspark.sql.functions as F

In [ ]:
salt_factor = 5 # Exemplo: Adicionar 5 valores Salting diferentes( valores entre 0 e 4)

In [ ]:
# Adicionaldo o salt apenas em df2(Maior Dataframe)
df2_with_salt =df2.withColumn("salt",(rand()* salt_factor).cast("int")) # criando uma coluna nova para acrescentar o salting
display(df2_with_salt)

Expandindo o df1 (dataframe menor), para incluir todas as combinações possíveis de salting
no dataframe menor, você cria na coluna salt, todas as chaves aleatórias, para que no momento em que se for
fazer o join, exista a referência ( código ) nos dois dataframes

In [ ]:
df1_expanded = df1.crossJoin(spark.range(salt_factor).toDF("salt"))
display (df1_expanded)

COMMAND ----------

MAGIC %md
MAGIC AUMENTANDO O NÚMERO DE PARTIÇÕES PARA BALANCEAR O PARTICIONAMENTO SHUFFLE.PARTITIONS

COMMAND ----------

comando para aumentar o número de partições
PARA EXECUTAR ESSA TÉCNICA, SERÁ NECESSÁRIO REALIZAR UM RESET RÁPIDO

In [ ]:
spark.conf.set('spark.sql.shuffle.partitions','500')

COMMAND ----------

MAGIC %md
MAGIC à partir do spark 3.0 adaptative query execution
MAGIC ajustar automaticamente a quantidade de suffle partitions, baseada no tamanho e estágio do processamento

COMMAND ----------

deixa o spark habilitado para o spark ajudar a otimizar no balanceamento do particionamento

In [ ]:
spark.conf.set("spark.sql.adaptative.enabled","true")